In [0]:
service_credential = dbutils.secrets.get(scope="ecomm-kv-scope",key="ecomm-secret")

spark.conf.set("fs.azure.account.auth.type.ecommadls.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.ecommadls.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.ecommadls.dfs.core.windows.net", "3b059ba0-2363-42d4-345890-37UC3dd866ea7")
spark.conf.set("fs.azure.account.oauth2.client.secret.ecommadls.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.ecommadls.dfs.core.windows.net", "https://login.microsoftonline.com/2535435251-7BG9-4c72-905f-39bf434340a8a84/oauth2/token")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name, current_timestamp, col

In [0]:
root = "abfss://zone-2@ecommadls.dfs.core.windows.net"

In [0]:
display(dbutils.fs.ls(root))

path,name,size,modificationTime
abfss://zone-2@ecommadls.dfs.core.windows.net/bronze/,bronze/,0,1756261104000
abfss://zone-2@ecommadls.dfs.core.windows.net/buyers-raw-2/,buyers-raw-2/,0,1756227826000
abfss://zone-2@ecommadls.dfs.core.windows.net/countries-raw-2/,countries-raw-2/,0,1756227844000
abfss://zone-2@ecommadls.dfs.core.windows.net/gold/,gold/,0,1756261215000
abfss://zone-2@ecommadls.dfs.core.windows.net/sellers-raw-2/,sellers-raw-2/,0,1756227835000
abfss://zone-2@ecommadls.dfs.core.windows.net/silver/,silver/,0,1756261150000
abfss://zone-2@ecommadls.dfs.core.windows.net/to_processed_user_data/,to_processed_user_data/,0,1756319717000
abfss://zone-2@ecommadls.dfs.core.windows.net/users-raw-2/,users-raw-2/,0,1756227817000


In [0]:
display(dbutils.fs.ls(f"{root}/sellers-raw-2"))

path,name,size,modificationTime
abfss://zone-2@ecommadls.dfs.core.windows.net/sellers-raw-2/Comparison-of-Sellers-by-Gender-and-Country.parquet,Comparison-of-Sellers-by-Gender-and-Country.parquet,9946,1756318371000


In [0]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [0]:
spark

In [0]:
bronze_path = f"{root}/bronze"

In [0]:
# Read from raw
userDF = spark.read.parquet(f"{root}/users-raw-2")

# Write to Bronze in Delta format
userDF.write.format("delta") \
    .mode("overwrite") \
    .save(f"{bronze_path}/users")

In [0]:
buyersDF = spark.read.parquet(f"{root}/buyers-raw-2")

buyersDF.write.format("delta") \
    .mode("overwrite") \
    .save(f"{bronze_path}/buyers")

In [0]:
sellersDF = spark.read.parquet(f"{root}/sellers-raw-2")

sellersDF.write.format("delta") \
    .mode("overwrite") \
    .save(f"{bronze_path}/sellers")

In [0]:
countriesDF = spark.read.parquet(f"{root}/countries-raw-2")

countriesDF.write.format("delta") \
    .mode("overwrite") \
    .save(f"{bronze_path}/countries")